### Baharestan Statistics,Pockets,site_sandali : use bahar_sandali_All.xlsx
#### result is : bahar_prep1.xlsx

In [1]:
import pandas as pd
import os
import shutil
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter

data = pd.read_excel('bahar_sandali_All.xlsx')  
df = pd.DataFrame(data)  


df['tedad'] = 1

# گروه‌بندی و جمع‌آوری داده‌ها
result = df.groupby(['makan','code']).agg(
    tedad2=('tedad', 'sum'),
    sandaliFirst=('sandali', 'min'),
    sandaliEnd=('sandali', 'max'),
    tarikh=('tarikh','first'),
    saat=('saat','first'),
    #makan=('makan','first'),
    noe=('noe','first'),
    dars=('dars','first')

    
).reset_index()

#result = result.merge(df[['code', 'dars','noe','tarikh','saat']], on='code', how='left')
result = result.sort_values(by=['tarikh', 'saat','makan'])

result.to_excel('bahar_prep1.xlsx', index=False)

print("DataFrame successfully saved to bahar_prep1.xlsx")


DataFrame successfully saved to bahar_prep1.xlsx


### Baharestan Generate Pockets : use bahar_prep1.xlsx

In [2]:
import pandas as pd
from docx import Document
from docx.shared import Inches
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

# ایجاد سند ورد
doc = Document()

# ستون‌ها و ترتیب نمایشی آنها (از راست به چپ)
columns_to_include = ['dars', 'noe', 'saat', 'tarikh', 'tedad2', 'code', 'makan']
columns_to_include_rtl = columns_to_include[::-1]

# خواندن فایل اکسل فقط با ستون‌های موردنظر
df = pd.read_excel('bahar_prep1.xlsx', usecols=columns_to_include)
df = df[columns_to_include]

# تابع تنظیم راست‌چین بودن پاراگراف
def set_rtl_paragraph(cell, text, spacing=2.2):
    para = cell.paragraphs[0]
    para.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
    para.paragraph_format.line_spacing = spacing
    para.text = text

    pPr = para._element.get_or_add_pPr()
    bidi = OxmlElement('w:bidi')
    pPr.append(bidi)

# عرض ستون‌ها (در اینجا به همان صورت تعریف‌شده نگه داشته‌ایم)
column_widths_px = {
    'dars': 290,
    'noe': 200,
    'saat': 80,
    'tarikh': 100,
    'tedad2': 40,
    'code': 80,
    'makan': 190
}
column_widths_inch = {col: Inches(px / 96) for col, px in column_widths_px.items()}

def set_col_widths(table, widths):
    for row in table.rows:
        for idx, width in enumerate(widths):
            cell = row.cells[idx]
            tc = cell._tc
            tcPr = tc.get_or_add_tcPr()
            tcW = OxmlElement('w:tcW')
            tcW.set(qn('w:type'), 'dxa')
            tcW.set(qn('w:w'), str(int(width.inches * 1440)))
            tcPr.append(tcW)

# گروه‌بندی و تولید جداول
current_makan = None
current_group = []

for idx, row in df.iterrows():
    if current_makan is None:
        current_makan = row['makan']

    if row['makan'] != current_makan:
        if current_group:
            doc.add_heading(f'{current_makan}', level=1)
            table = doc.add_table(rows=1, cols=len(columns_to_include))
            table.style = 'Table Grid'

            # ردیف عنوان (راست به چپ)
            for j, col_name in enumerate(columns_to_include_rtl):
                cell = table.cell(0, j)
                set_rtl_paragraph(cell, col_name)

            # ردیف‌های داده
            for item in current_group:
                row_cells = table.add_row().cells
                for j, col_name in enumerate(columns_to_include_rtl):
                    set_rtl_paragraph(row_cells[j], str(item[columns_to_include.index(col_name)]))

            set_col_widths(table, [column_widths_inch[col] for col in columns_to_include_rtl])
            doc.add_page_break()

        current_makan = row['makan']
        current_group = [row.tolist()]
    else:
        current_group.append(row.tolist())

# اضافه کردن آخرین گروه
if current_group:
    doc.add_heading(f'{current_makan}', level=1)
    table = doc.add_table(rows=1, cols=len(columns_to_include))
    table.style = 'Table Grid'

    for j, col_name in enumerate(columns_to_include_rtl):
        cell = table.cell(0, j)
        set_rtl_paragraph(cell, col_name)

    for item in current_group:
        row_cells = table.add_row().cells
        for j, col_name in enumerate(columns_to_include_rtl):
            set_rtl_paragraph(row_cells[j], str(item[columns_to_include.index(col_name)]))

    set_col_widths(table, [column_widths_inch[col] for col in columns_to_include_rtl])

# ذخیره نهایی
output_path = "Baharestan_pocket_RTL.docx"
doc.save(output_path)
print("Saved to:", output_path)


Saved to: Baharestan_pocket_RTL.docx
